In [1]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForMaskedLM, BertweetTokenizer

config = AutoConfig.from_pretrained('vinai/bertweet-base')
tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base')
# model = AutoModelForMaskedLM.from_config(config)
model = AutoModelForMaskedLM.from_pretrained('vinai/bertweet-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from glob import glob
from datasets import Dataset

import re
import pandas as pd

from parse import mask_data_loading

data_url = '../crawler/stock/data/**.json'
url = glob(data_url)[-1]
data, symbols = mask_data_loading(url, tokenizer)

dataset = Dataset.from_pandas(data.loc[:, ['labels', 'sentense']])
dataset = dataset.remove_columns('__index_level_0__')
dataset = dataset.train_test_split(test_size=0.2)

train_dataset = dataset['train']
test_dataset = dataset['test']

train_dataset = train_dataset.shuffle().select(range(50000))

special_tokens_dict = {'additional_special_tokens': list(symbols)}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(65862, 768)

In [3]:
tokenizer.save_pretrained('./symbol-vocab')

('./symbol-vocab\\tokenizer_config.json',
 './symbol-vocab\\special_tokens_map.json',
 './symbol-vocab\\vocab.txt',
 './symbol-vocab\\bpe.codes',
 './symbol-vocab\\added_tokens.json')

In [4]:
def encode(example):
    sentense = example['sentense']
    label = example['labels']

    label = tokenizer(label, padding='max_length', truncation=True)
    # 101, 51, 1234, 12541, 151
    result = tokenizer(sentense, padding='max_length', truncation=True)
    # 101, 103, 103, 103
    result['label_ids'] = label['input_ids']

    # masked_position = [i for i in range(len(result['input_ids'])) if result['input_ids'][i] == tokenizer.mask_token_id]
    # result['decoder_input_ids'] = label['input_ids']
    # result['labels'] = [-100 for i in label['input_ids']]
    # for i in range(len(result['labels'])):
    #     if not i in masked_position:
    #         result['labels'][i] = -100

    return result

encoded_train_dataset = train_dataset.map(encode, batched=True)
encoded_test_dataset = test_dataset.map(encode, batched=True)

# print(encoded_dataset[0]['sentense'])
# print(encoded_dataset[0]['input_ids'])
# print(encoded_dataset[0]['labels'])
# print(encoded_dataset[0]['label_ids'])
# print(tokenizer.ids_to_tokens)
# print(tokenizer.decode(encoded_dataset[0]['label_ids']))

100%|██████████| 28/28 [00:29<00:00,  1.06s/ba]


# Training

In [5]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [6]:
from transformers import TrainingArguments, Trainer, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir="after-bert-trainer",
    per_device_train_batch_size=4,
    num_train_epochs=5,
    save_steps=50000
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: sentense. If sentense are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
C:\Users\douli\anaconda3\envs\stock-nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50000
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 62500
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB

  1%|          | 501/62500 [01:06<3:07:59,  5.50it/s]

{'loss': 0.3378, 'learning_rate': 4.96e-05, 'epoch': 0.04}


  2%|▏         | 1001/62500 [02:11<3:13:03,  5.31it/s]

{'loss': 0.0689, 'learning_rate': 4.92e-05, 'epoch': 0.08}


...

 99%|█████████▉| 62001/62500 [2:13:26<01:30,  5.49it/s]

{'loss': 0.0177, 'learning_rate': 4.0000000000000003e-07, 'epoch': 4.96}


100%|██████████| 62500/62500 [2:14:30<00:00,  4.93it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 62500/62500 [2:14:30<00:00,  7.74it/s]

{'loss': 0.0198, 'learning_rate': 0.0, 'epoch': 5.0}
{'train_runtime': 8073.667, 'train_samples_per_second': 30.965, 'train_steps_per_second': 7.741, 'train_loss': 0.06624277964782715, 'epoch': 5.0}


TrainOutput(global_step=62500, training_loss=0.06624277964782715, metrics={'train_runtime': 8073.667, 'train_samples_per_second': 30.965, 'train_steps_per_second': 7.741, 'train_loss': 0.06624277964782715, 'epoch': 5.0})

In [7]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
    device=0
)

s = f'{tokenizer.mask_token} to the sky!!!'
fill_mask(s)

[{'score': 0.9999397993087769,
  'token': 65345,
  'token_str': '$ T S L A',
  'sequence': 'to the sky!!!'},
 {'score': 1.6432995835202746e-05,
  'token': 64149,
  'token_str': '$ A A P L',
  'sequence': 'to the sky!!!'},
 {'score': 8.08750519354362e-06,
  'token': 65545,
  'token_str': '$ A M Z N',
  'sequence': 'to the sky!!!'},
 {'score': 6.454405593103729e-06,
  'token': 65659,
  'token_str': '$ F B',
  'sequence': 'to the sky!!!'},
 {'score': 3.896896942023886e-06,
  'token': 65350,
  'token_str': '$ N V D A',
  'sequence': 'to the sky!!!'}]

In [8]:
import torch
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    2097 MB |    2892 MB |  208162 GB |  208160 GB |
|       from large pool |    2093 MB |    2869 MB |  206862 GB |  206860 GB |
|       from small pool |       4 MB |      24 MB |    1300 GB |    1300 GB |
|---------------------------------------------------------------------------|
| Active memory         |    2097 MB |    2892 MB |  208162 GB |  208160 GB |
|       from large pool |    2093 MB |    2869 MB |  206862 GB |